In [1]:
import os

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from lared_laughter.annotation.coco_reader import CocoReader
from lared.dataset.example import VideoExample, AudioExample, FullExample
from lared.data_loading.utils import get_video_caps, AudioFiles
from lared_laughter.constants import raw_audio_path, raw_video_path, processed_data_path, annot_exp_path, datasets_path

In [4]:
VideoExample.init_caps(get_video_caps(raw_video_path))
VideoExample.init_camera_annotation(os.path.join(processed_data_path, 'elan_out/camera.txt'))
AudioExample.init_dataset(AudioFiles(raw_audio_path))

# Laughter examples: loading
load a previously generated set of examples

In [5]:
# read the inputs
# laughter_examples_df = pd.read_csv(os.path.join(cloud_data_path, 'laughter_data', 'laughter_examples', 'examples.csv'), index_col=0)
# laughter_examples = {ex[1].hash: FullExample(**ex[1].to_dict()) for ex in laughter_examples_df.iterrows()}
# speech_examples_df = pd.read_csv(os.path.join(cloud_data_path, 'laughter_data', 'speech_examples', 'examples.csv'), index_col=0)
# speech_examples = {ex[1].hash: FullExample(**ex[1].to_dict()) for ex in speech_examples_df.iterrows()}

laughter_bbs = CocoReader(os.path.join(annot_exp_path, 'laughter_examples', 'cvat.json'))
speech_bbs = CocoReader(os.path.join(annot_exp_path, 'speech_examples', 'cvat.json'))

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/c/Users/Jose/gdrive/data/lared_laughter/annotation_experiment_2/speech_examples/cvat.json'

In [14]:
examples_df = pd.read_csv(os.path.join(annot_exp_path, 'processed', 'examples_without_calibration.csv'))

In [25]:
examples_df[examples_df['hash'] == '0a8b94fbb8407ed3f108f84008be0579946af245a11e656a57458a9e6db69b71'].head(1)

,Unnamed: 0,person,cam,rect,instance_id,condition,calibration,hash,ini_time,end_time,...,is_laughter,confidence,intensity,has_continuous,attempt,pressed_key,onset,offset,G,N
420,3840,25,1,"[1460, 702, 186, 238]",2_c9c20f2eee48c8f1f8120bb25055b3bbb167d1f9284f...,video,False,0a8b94fbb8407ed3f108f84008be0579946af245a11e65...,2329.000869,2338.546822,...,False,5.0,4.0,True,0,False,NaN,NaN,5,3


In [16]:
examples = examples_df.groupby('hash').agg({
    'hash': 'first',
    'cam': 'first',
    'rect': 'first',
    'person': 'first',
    'ini_time': 'first',
    'end_time': 'first',
    '_ini_time': 'first',
    '_end_time': 'first'
}).rename(columns={'person':'pid'})

In [17]:
examples = {ex[1].hash: FullExample(**ex[1].to_dict()) for ex in examples.iterrows()}

In [20]:
def write_dataset():
    # output paths
    out_path = os.path.join(datasets_path, 'tight')
    audio_path = os.path.join(out_path, 'audio')
    video_path = os.path.join(out_path, 'video')
    av_path = os.path.join(out_path, 'av')
    if not os.path.exists(audio_path): os.makedirs(audio_path)
    if not os.path.exists(video_path): os.makedirs(video_path)
    if not os.path.exists(av_path): os.makedirs(av_path)

    for hash, ex in tqdm(examples.items()):
        # if hash not in laughter_bbs and hash not in speech_bbs:
        #     continue
        # ex.rect = laughter_bbs[hash] if hash in laughter_bbs else speech_bbs[hash]
        # if not ex.rect:
        #     continue
        ex.write_audio(audio_path, fname=hash, padded=False)
        ex.write_video(video_path, fname=hash, padded=False, enlarge_perc=0)
        # ex.write_audiovisual(video_path, audio_path, av_path, fname=hash)

In [21]:
write_dataset()

  0%|          | 0/628 [00:00<?, ?it/s]